In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
from pandas import json_normalize
from requests.exceptions import HTTPError
from requests.auth import HTTPBasicAuth
from getpass import getpass
pd.set_option('display.max_rows', None)

Have the EC2 ping the API and create an S3 bucket

In [2]:
import sys
sys.path.append("../src")

In [3]:
import config
from sodapy import Socrata

client = Socrata('data.cityofnewyork.us',
                     config.api_key,
                     config.api_username,
                     config.api_password)

# First 5000 results, returned as JSON from API / converted to
# Python list of dictionaries by sodapy
results = client.get("2upf-qytp", limit=5000)

# Convert to pandas DataFrame and get a smaller sample to try out functions
results_df = pd.DataFrame.from_records(results)
df = results_df.sample(frac=.2)

In [4]:
df.index.rename('Transaction Number', inplace=True)

In [5]:
df = df.rename(columns={'vendorid': 'Vendor Id'
                   ,'tpep_pickup_datetime':'Pickup Datetime'
                   ,'tpep_dropoff_datetime': 'Dropoff Datetime'
                   ,'passenger_count':'Passenger Count'
                   , 'trip_distance': 'Trip Distance'
                   , 'ratecodeid': 'Rate Code Id'
                   , 'store_and_fwd_flag':'Store and Forward'
                   , 'pulocationid': 'Pick Up Location Id'
                   ,'dolocationid': 'Drop Off Location Id'
                   ,'payment_type': 'Payment Type'
                   , 'fare_amount': 'Fare Amount'
                   , 'extra': 'Extra'
                   ,'mta_tax': 'MTA Tax'
                   , 'tip_amount': 'Tip Amount'
                   , 'tolls_amount': 'Tolls Amount'
                   , 'improvement_surcharge': 'Improvement Surcharge'
                   ,'total_amount': 'Total Amount'
                   , 'congestion_surcharge':'Congestion Surcharge'})
                   
# df.head()        



In [6]:
df.columns

Index(['Vendor Id', 'Pickup Datetime', 'Dropoff Datetime', 'Passenger Count',
       'Trip Distance', 'Rate Code Id', 'Store and Forward',
       'Pick Up Location Id', 'Drop Off Location Id', 'Payment Type',
       'Fare Amount', 'Extra', 'MTA Tax', 'Tip Amount', 'Tolls Amount',
       'Improvement Surcharge', 'Total Amount', 'Congestion Surcharge'],
      dtype='object')

In [7]:
cols = ['Pick Up Location Id', 'Drop Off Location Id', 'Passenger Count', 'Trip Distance', 
        'Fare Amount', 'Extra', 'MTA Tax', 'Tip Amount', 'Tolls Amount',
       'Improvement Surcharge', 'Total Amount', 'Congestion Surcharge']

for col in cols:
    df[col] = pd.to_numeric(df[col])

In [8]:
# datetimes are datetimes
df['Pickup Datetime'] = pd.to_datetime(df['Pickup Datetime'])

In [9]:
# datetimes are datetimes
df['Dropoff Datetime'] = pd.to_datetime(df['Dropoff Datetime'])

In [10]:
df['Trip Time (hrs)']=((df['Dropoff Datetime'] - df['Pickup Datetime'])/np.timedelta64(1, 'h')).round(2)

In [11]:
# engineer feature: mph (time / distance )
df['Trip Speed mph'] = (df['Trip Distance'] / df['Trip Time (hrs)']).round(2)


In [12]:
# replace 0 with "None", 0.5 with "Rush Hour"
# df['Extra'] = df['Extra'].replace({'0.5':'Rush Hour', '1': 'Overnight', '0': 'None'})

In [13]:
df['Rate Code Id'] = df['Rate Code Id'].replace({'1':'Standard rate',
                                                 '2':'JFK', 
                                                 '3':'Newark',
                                                 '4':'Nassau or Westchester',
                                                 '5':'Negotiated fare',
                                                 '6':'Group ride'})

In [14]:
df['Store and Forward'] = df['Store and Forward'].replace({'Y':True, 'N':False})

In [15]:
df['Payment Type'] = df['Payment Type'].replace({'1':'Credit Card'
                                                 , '2':'Cash'
                                                 , '3':'No Charge'
                                                 , '4':'Dispute'
                                                 , '5':'Unknown'
                                                 , '6':'Voided_Trip'})

In [16]:
df['Vendor Id'] = df['Vendor Id'].replace({'1':'Creative Mobole'
                                           , '2':'VeriFone'})

In [17]:
df['Congestion Surcharge'].unique()

array([ 2.5,  0. , -2.5])

In [18]:
df = pd.get_dummies(df, columns=['Payment Type'], dummy_na=True, drop_first=False)

In [21]:
taxi_zones = pd.read_csv('../data/taxi+_zone_lookup.csv')


In [22]:
#left join df wtih taxi_zones on df[Pick Up Location Id]== taxi_zones[LocationId] 
# and df[Drop Off Location ID] == taxi_zone[LocationID]
taxi_zones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [23]:
df = df.merge(taxi_zones, how='left', left_on='Pick Up Location Id',  right_on='LocationID', suffixes=('_Pick_Up', '_Pick_Up_Zone_name'))

In [24]:
df = df.drop('LocationID', axis=1)

In [25]:
df = df.merge(taxi_zones, how='left', left_on='Drop Off Location Id',  right_on='LocationID', suffixes=('_PickUp', '_DropOff'))



In [26]:
df = df.drop('LocationID', axis=1)

In [27]:
df.head()

,Vendor Id,Pickup Datetime,Dropoff Datetime,Passenger Count,Trip Distance,Rate Code Id,Store and Forward,Pick Up Location Id,Drop Off Location Id,Fare Amount,...,Payment Type_Credit Card,Payment Type_Dispute,Payment Type_No Charge,Payment Type_nan,Borough_PickUp,Zone_PickUp,service_zone_PickUp,Borough_DropOff,Zone_DropOff,service_zone_DropOff
0,VeriFone,2019-09-26 20:21:43,2019-09-26 20:42:22,1,1.91,Standard rate,False,234,170,13.0,...,1,0,0,0,Manhattan,Union Sq,Yellow Zone,Manhattan,Murray Hill,Yellow Zone
1,Creative Mobole,2019-09-26 20:23:44,2019-09-26 20:28:00,3,0.50,Standard rate,False,141,237,4.5,...,0,0,0,0,Manhattan,Lenox Hill West,Yellow Zone,Manhattan,Upper East Side South,Yellow Zone
2,VeriFone,2019-09-26 21:47:34,2019-09-26 21:52:18,1,0.70,Standard rate,False,144,45,5.0,...,1,0,0,0,Manhattan,Little Italy/NoLiTa,Yellow Zone,Manhattan,Chinatown,Yellow Zone
3,VeriFone,2019-09-26 20:11:42,2019-09-26 20:19:32,3,1.53,Standard rate,False,186,113,7.5,...,0,0,0,0,Manhattan,Penn Station/Madison Sq West,Yellow Zone,Manhattan,Greenwich Village North,Yellow Zone
4,Creative Mobole,2019-09-26 20:44:31,2019-09-26 20:58:48,3,1.50,Standard rate,False,142,68,10.5,...,0,0,0,0,Manhattan,Lincoln Square East,Yellow Zone,Manhattan,East Chelsea,Yellow Zone


In [28]:
df.columns

Index(['Vendor Id', 'Pickup Datetime', 'Dropoff Datetime', 'Passenger Count',
       'Trip Distance', 'Rate Code Id', 'Store and Forward',
       'Pick Up Location Id', 'Drop Off Location Id', 'Fare Amount', 'Extra',
       'MTA Tax', 'Tip Amount', 'Tolls Amount', 'Improvement Surcharge',
       'Total Amount', 'Congestion Surcharge', 'Trip Time (hrs)',
       'Trip Speed mph', 'Payment Type_Cash', 'Payment Type_Credit Card',
       'Payment Type_Dispute', 'Payment Type_No Charge', 'Payment Type_nan',
       'Borough_PickUp', 'Zone_PickUp', 'service_zone_PickUp',
       'Borough_DropOff', 'Zone_DropOff', 'service_zone_DropOff'],
      dtype='object')

In [29]:
reordered_columns = ['Vendor Id', 'Pickup Datetime', 'Dropoff Datetime', 'Passenger Count',
                'Trip Distance',  'Trip Time (hrs)', 'Trip Speed mph',
                'Rate Code Id', 'Store and Forward',
                'Pick Up Location Id', 'Borough_PickUp', 'Zone_PickUp', 'service_zone_PickUp',
                'Drop Off Location Id', 'Borough_DropOff', 'Zone_DropOff', 'service_zone_DropOff', 
                'Fare Amount', 'Extra', 'MTA Tax', 'Tip Amount', 'Tolls Amount', 'Improvement Surcharge',
                'Total Amount', 'Congestion Surcharge', 'Payment Type_Cash', 'Payment Type_Credit Card',
                'Payment Type_No Charge', 'Payment Type_nan', 'Payment Type_Dispute']


In [34]:
df_final = df.reindex(columns=reordered_columns)

In [36]:
df_final.head()

,Vendor Id,Pickup Datetime,Dropoff Datetime,Passenger Count,Trip Distance,Trip Time (hrs),Trip Speed mph,Rate Code Id,Store and Forward,Pick Up Location Id,...,Tip Amount,Tolls Amount,Improvement Surcharge,Total Amount,Congestion Surcharge,Payment Type_Cash,Payment Type_Credit Card,Payment Type_No Charge,Payment Type_nan,Payment Type_Dispute
0,VeriFone,2019-09-26 20:21:43,2019-09-26 20:42:22,1,1.91,0.34,5.62,Standard rate,False,234,...,3.36,0.0,0.3,20.16,2.5,0,1,0,0,0
1,Creative Mobole,2019-09-26 20:23:44,2019-09-26 20:28:00,3,0.50,0.07,7.14,Standard rate,False,141,...,0.00,0.0,0.3,8.30,2.5,1,0,0,0,0
2,VeriFone,2019-09-26 21:47:34,2019-09-26 21:52:18,1,0.70,0.08,8.75,Standard rate,False,144,...,1.00,0.0,0.3,9.80,2.5,0,1,0,0,0
3,VeriFone,2019-09-26 20:11:42,2019-09-26 20:19:32,3,1.53,0.13,11.77,Standard rate,False,186,...,0.00,0.0,0.3,11.30,2.5,1,0,0,0,0
4,Creative Mobole,2019-09-26 20:44:31,2019-09-26 20:58:48,3,1.50,0.24,6.25,Standard rate,False,142,...,0.00,0.0,0.3,14.30,2.5,1,0,0,0,0


### Did
- reindex so each row is a trip instance CHECK
- datetimes are datetimes CHECK
- engineer feature: mph (time / distance ) CHECK
- Rename extra to none, overnight, rush hour CHECK 
- convert all numeric strings to floats/numerics CHECK
- match taxizone with zipcode or location name CHECK
- Convert Rate Code id to 1= Standard rate, 2=JFK, 3=Newark, 4=Nassau or Westchester  5=Negotiated fare, 6=Group ride
- one hot encode the payment types 
-  reorder the taxicode pickup and drop off names



### To Do
- Rename the suffixed columns headers